# Домашнее задание

1. Взять 1 любой полученный список векторов из предыдущей лабы
2. Разделить данные на обучающее и тестовое множество
3. Выбрать метрику для релевантности поста (лайки/репосты/...). Сгруппировать все данные в несколько классов (популярный/непопулярный пост).
4. Обучить минимум 3 классификатора
5. Оценить их работу с помощью метрик
6. Выбрать наилучший классификатор

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Загрузка датафрейма всех постов паблика ИТМО в ВК
df = pd.read_csv("/content/drive/MyDrive/preprocessed_data_soobshchestva_itmo.csv")
df.head()

,id_group,id_post,data,description,title,text,views,likes,reposts,processed_text
0,-94,57196,2024-04-12 13:00:03,NaN,NaN,Пятница! А где тусуются айтишники? 👇,7473.0,11,6,пятница тусоваться айтишник
1,-94,57194,2024-04-12 09:00:02,NaN,NaN,"Каждый из нас, своего рода, художник 👨🏻‍🎨 \n \...",1542.0,3,2,каждый свой род художник особенно наш друг сев...
2,-94,57190,2024-04-11 09:00:01,NaN,NaN,"Мы его ждали и он возвращается! Грандиозный, л...",3634.0,89,55,ждать возвращаться грандиозный летний свежий i...
3,-94,57182,2024-04-10 12:39:06,Где узнать больше о DevOps и IT в целом? 13 ап...,Клип @itmoru,Тут про свежую IT-конференцию👇🏻\n\nВ ролике Вя...,3703.0,18,11,свежий it-конференция ролик вячеслав евстрат s...
4,-94,57181,2024-04-10 11:41:01,NaN,NaN,Прекрасные весенние новости! ИТМО — лучший рос...,4597.0,106,18,прекрасный весенний новость итмый хороший росс...


In [ ]:
# Удаление строк, которые не являются строками, из столбца 'processed_text', чтобы совподали длины векторов с количеством столбцов
df = df.drop(df[~df['processed_text'].apply(lambda x: isinstance(x, str))].index)
df.head()

,id_group,id_post,data,description,title,text,views,likes,reposts,processed_text
0,-94,57196,2024-04-12 13:00:03,NaN,NaN,Пятница! А где тусуются айтишники? 👇,7473.0,11,6,пятница тусоваться айтишник
1,-94,57194,2024-04-12 09:00:02,NaN,NaN,"Каждый из нас, своего рода, художник 👨🏻‍🎨 \n \...",1542.0,3,2,каждый свой род художник особенно наш друг сев...
2,-94,57190,2024-04-11 09:00:01,NaN,NaN,"Мы его ждали и он возвращается! Грандиозный, л...",3634.0,89,55,ждать возвращаться грандиозный летний свежий i...
3,-94,57182,2024-04-10 12:39:06,Где узнать больше о DevOps и IT в целом? 13 ап...,Клип @itmoru,Тут про свежую IT-конференцию👇🏻\n\nВ ролике Вя...,3703.0,18,11,свежий it-конференция ролик вячеслав евстрат s...
4,-94,57181,2024-04-10 11:41:01,NaN,NaN,Прекрасные весенние новости! ИТМО — лучший рос...,4597.0,106,18,прекрасный весенний новость итмый хороший росс...


In [ ]:
import numpy as np

# Загрузка данных векторов методом Word2Vec из файла
vectors_wv = np.load("/content/drive/MyDrive/word2vec_vectors.npy")

# Вывод первых строк
print(vectors_wv[:5])

[[-0.06475276  0.00477081  0.01552755 ...  0.04893025 -0.04911068
   0.01917364]
 [ 0.01883718 -0.00407822  0.00325054 ...  0.01734087 -0.0292731
  -0.01256763]
 [ 0.00117679  0.01816967 -0.00049315 ...  0.02281608 -0.02196103
  -0.00239539]
 [ 0.01353688  0.01516888 -0.01883488 ...  0.01475182 -0.03057989
  -0.01262961]
 [ 0.01576324 -0.00426953 -0.01952407 ...  0.00791807 -0.04074295
  -0.01524256]]


In [ ]:
from sklearn.model_selection import train_test_split

# Разделение данных
X = vectors_wv
y = (df['likes'] > df['likes'].median()).astype(int)  # Популярный пост, если просмотры выше медианы

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Логистическая регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
#классификатор
clf = LogisticRegression(
        penalty = 'l2', #{‘l1’, ‘l2’, ‘elasticnet’, None}, default=’l2’
        C = 0.2, #float, default=1.0
        class_weight = 'balanced', #dict or ‘balanced’, default=None
        random_state = 42,
        max_iter = 100, #int, default=100
        n_jobs = -1, #int, default=None
)

In [ ]:
#предобученный классификатор
clf_pretrained = clf.fit(X_train, y_train)

In [ ]:
#результат предсказания
y_predict = clf_pretrained.predict(X_test)
print(y_predict)
print(y_test)

[1 0 0 ... 0 0 1]
6272     0
1293     1
4469     1
296      1
2892     1
        ..
13877    0
730      1
13035    0
12166    0
1140     1
Name: likes, Length: 2811, dtype: int64


In [ ]:
from sklearn.metrics import classification_report

Можем увидеть, что точность логистической регрессии составялет 0.7449

In [ ]:
print(classification_report(y_test, y_predict, digits = 4))

              precision    recall  f1-score   support

           0     0.7863    0.6871    0.7334      1435
           1     0.7116    0.8052    0.7555      1376

    accuracy                         0.7449      2811
   macro avg     0.7490    0.7462    0.7444      2811
weighted avg     0.7497    0.7449    0.7442      2811



# Случайный лес



In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf_rf =  RandomForestClassifier(
        n_estimators = 200, #int, default=100
        criterion = 'gini', #{“gini”, “entropy”, “log_loss”}, default=”gini”
        max_depth = 200, #int, default=None
        class_weight = 'balanced', #dict or ‘balanced’, default=None
        random_state = 42
)

In [ ]:
#предобученный классификатор
clf_rf = clf_rf.fit(X_train, y_train)
y_predict_rf = clf_rf.predict(X_test)

Можем увидеть, что точность классификатора случайный лес составялет 0.8111

In [ ]:
print(classification_report(y_test, y_predict_rf, digits = 4))

              precision    recall  f1-score   support

           0     0.8100    0.8230    0.8165      1435
           1     0.8123    0.7987    0.8054      1376

    accuracy                         0.8111      2811
   macro avg     0.8111    0.8108    0.8109      2811
weighted avg     0.8111    0.8111    0.8111      2811



# K-ближайших соседей



In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
clf_KNN = KNeighborsClassifier(
        n_neighbors = 3, #int, default=5
        weights = 'distance', #{‘uniform’, ‘distance’}, callable or None, default=’uniform’
        metric = 'euclidean', #str or callable, default=’minkowski’
)

In [ ]:
clf_KNN = clf_KNN.fit(X_train, y_train)
y_predict_KNN = clf_KNN.predict(X_test)

Можем увидеть, что точность классификатора случайный лес составялет 0.7254

In [ ]:
print(classification_report(y_test, y_predict_KNN, digits = 4))

              precision    recall  f1-score   support

           0     0.8435    0.5672    0.6783      1435
           1     0.6636    0.8903    0.7604      1376

    accuracy                         0.7254      2811
   macro avg     0.7536    0.7288    0.7194      2811
weighted avg     0.7554    0.7254    0.7185      2811



#ИТОГ
Самый лучший классификатор по метрикам accuracy и f1_score - случайный лес с точностью 0.811 и 0.810

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
print('Логистическая регрессия',f1_score(y_test, y_predict, average='macro'))
print('Случайный лес',f1_score(y_test, y_predict_rf, average='macro'))
print('K-ближайших соседей',f1_score(y_test, y_predict_KNN, average='macro'))

Логистическая регрессия 0.7444492640078653
Случайный лес 0.810938370210907
K-ближайших соседей 0.7193653010552452


# Лабораторная работа 7

1. Взять любые вектора W2V
2. Взять наилучший классификатор из предыдущей лабы Случайный лес
3. Подобрать к нему гиперпараметры любым методом HalvingGridSearch
4. Выполнить предсказания популярности нового поста

In [ ]:
#Разделяем данные на тренировочный набор, проверочный и тестовый
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.50, random_state=42)

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.pipeline import Pipeline

In [ ]:
#определяем классификатор
pipeline = Pipeline([('clf', RandomForestClassifier())])

In [ ]:
# Определим сетку параметров для случайного леса
parameters = {
    'clf__n_estimators': [50, 100, 200],
    'clf__max_depth': [None, 50, 100, 200],
    'clf__min_samples_leaf': [1, 2, 4]
}

halving_search = HalvingGridSearchCV(
    estimator=pipeline,
    param_grid=parameters,
    cv = 5,
    factor=1.2,
    random_state=0,
)

In [ ]:
halving_search.fit(X_train, y_train)

In [ ]:
best_parameters = halving_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print(f"{param_name}: {best_parameters[param_name]}")

In [ ]:
y_pred_optimized = halving_search.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred_optimized))